In [ ]:
import sys
import os

In [ ]:
# include path to our library
cwd = os.getcwd()
dir_path = os.path.dirname(os.path.realpath(cwd))
sys.path.append(dir_path)

In [ ]:
dir_path
%cd $dir_path
!ls

In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from core.data.data_loader import *

In [ ]:
prefix = './dataset/2020_100_sorted.csv'  
df = pd.read_csv(prefix, header=0, engine='python')


hit = 19
begin_hit = 'hit_id_'
end_hit = 'value_'
begin = begin_hit+str(hit)
end = end_hit+str(hit)

df = df.iloc[:, 9:]
df.head()

In [ ]:
interval = 11
begin_cartesian = 1
end_cartesian = 4
begin_val = 10
end_val = 11
hit = 5
#df.iloc[:,np.r_[1:4, 1:5]]
begin_cols = [begin_cartesian+(interval*hit) for hit in range(0,hit)]
print(begin_cols)
end_cols = [end_cartesian+(interval*hit) for hit in range(0,hit)]
print(end_cols)
#df.iloc[:,np.r_[1:4,begin_cartesian+(interval*hit):end_cartesian+(interval*hit)]]
new_df = None
for c in range(0,len(begin_cols)):
    frame = df.iloc[:,np.r_[begin_cols[c]:end_cols[c]]]
    new_df = pd.concat([new_df, frame], axis=1)
    
new_df.iloc[:,-3:]

In [ ]:
prefix = './dataset/2020_100_sorted.csv'  
data = Dataset(prefix, KindNormalization.Zscore)

#x, y = data.get_training_data(normalise=False, cylindrical=False, hit=10)
X = data.get_training_data(normalise=False, cylindrical=False, hit=10)
#X, y = data.convert_to_supervised(x.values, n_hit_in=4, n_hit_out=1,n_features=3)


In [ ]:

#x, y = X.iloc[0,0:3], X.iloc[0,12:15]
#x
X.head(10)

In [ ]:
def convert_to_supervisedd(sequences, n_hit_in, n_hit_out, n_features):
    '''
        n_hit_in : 3 number of hits
        n_hit_out: 1 number of future hits
        n_features 3
    '''
    X , Y = [],[]

    rows = sequences.shape[0]
    cols = sequences.shape[1]

    print(rows, cols)

    #seq_x, seq_y = sequences[0, 0:n_hit_in*n_features], sequences[0, n_hit_in*n_features:out_end_idx:]
    #X.append(seq_x)
    #Y.append(seq_y)
    for i in range(0, rows):

        for j in range(0, cols, n_features):
            #if j ==0:
            end_ix = j + n_hit_in*n_features
            out_end_idx = end_ix + n_hit_out*n_features
            #else:
            #    end_ix = j + n_hit_in
            #    out_end_idx = end_ix + n_hit_out*n_features
                
            if out_end_idx > cols+1:
                #print('corta ', out_end_idx)
                break
            #if i < 10:	
                #print('[%s,%s:%s][%s,%s:%s]' % (i, j, end_ix, i, end_ix, out_end_idx))
              
            seq_x, seq_y = sequences[i, j:end_ix], sequences[i, end_ix:out_end_idx]

            X.append(seq_x)
            Y.append(seq_y)

        # end_ix = n_hit_in*n_features
        # out_end_idx = end_ix + n_hit_out*n_features

        # 	#if out_end_idx > cols+1:
        # 	#	print('corta ', out_end_idx)
        # 	#	break
        # 	if i < 10:	
        # 		print('[%s,%s:%s][%s,%s:%s]' % (i,j,end_ix,i,end_ix,out_end_idx))		
        # 	seq_x, seq_y = sequences[i-1, n_features:end_ix].extend(sequences[i-1, end_ix:]), sequences[i, end_ix:out_end_idx]

        # 	X.append(seq_x)
        # 	Y.append(seq_y)

    return np.array(X) , np.array(Y)



#x, y = convert_to_supervisedd(X.values, n_hit_in=4, n_hit_out=1,n_features=3 )
Xdata, y = data.convert_to_supervised(X.values, n_hit_in=4, n_hit_out=1,n_features=3)

In [ ]:
X.iloc[:10,9:]

In [ ]:
Xdata.head(24)

In [ ]:
y.head(24)

In [ ]:
x_data = pd.DataFrame(x)
y_data = pd.DataFrame(y)

df2 = pd.concat([x_data, y_data],axis=1)
df2.head()